In [9]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from konlpy.tag import Twitter

from gensim.models.word2vec import Word2Vec
from os import path, mkdir
from multiprocessing import cpu_count

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from gensim.test.utils import common_texts,get_tmpfile

import re
import pickle
import time
from math import log10

In [ ]:
with open('artTokens.dat','rb') as file: # 미리 doc별로 저장한 tokens의 리스트를 load한다
    tokens = pickle.load(file)

In [4]:
#하이퍼파라미터 세팅
num_features = 100
min_word_count = 10
num_workers = cpu_count()
context_size = 10
downsampling = 1e-3
seed=180816

In [5]:
#모델 생성
art2vec =Word2Vec( 
    sg=1, #skip-gram
    workers=cpu_count(),
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample = downsampling,
)

In [44]:
art2vec.build_vocab(tokens)

In [45]:
art2vec.corpus_count

20514

In [46]:
art2vec.epochs

5

In [47]:
art2vec.train(tokens,total_examples=art2vec.corpus_count,epochs=art2vec.epochs) #학습 with tokens

(54894850, 74022550)

In [51]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
art2vec.save(r'''art2vec.w2v''')

In [8]:
##전에 학습한 결과 불러오기
art2vec = Word2Vec.load(r'''art2vec.w2v''')

In [9]:
art2vec.wv.most_similar('광주')

[('조진호', 0.7715893983840942),
 ('대구', 0.7689499855041504),
 ('부산', 0.7660964727401733),
 ('시립', 0.7494326829910278),
 ('전남', 0.7457484006881714),
 ('광역시', 0.7450919151306152),
 ('센텀시티', 0.7318888306617737),
 ('개점', 0.7264745235443115),
 ('대전', 0.7231205701828003),
 ('환기재단', 0.7210865616798401)]

In [10]:
art2vec.wv.most_similar('희망')

[('희망이', 0.8070540428161621),
 ('소망', 0.7373624444007874),
 ('행복', 0.7227489352226257),
 ('긍정', 0.7062973380088806),
 ('꿈', 0.7053670883178711),
 ('앞날', 0.7005017995834351),
 ('절망', 0.6873255968093872),
 ('염원', 0.6766011714935303),
 ('열망', 0.6744593381881714),
 ('기쁨', 0.6718753576278687)]

In [64]:
art2vec.wv.most_similar('가족','반려동물')

[('기억해야', 0.21745026111602783),
 ('콜렉터', 0.21343661844730377),
 ('생각합', 0.20056943595409393),
 ('블루칩', 0.1907409131526947),
 ('알려지', 0.19049102067947388),
 ('삼류', 0.18937496840953827),
 ('가요', 0.1706419587135315),
 ('박찬호', 0.17036212980747223),
 ('팝아티스트', 0.16764971613883972),
 ('연예인', 0.16761338710784912)]

In [56]:
art2vec.wv.most_similar('재미')

[('즐거움', 0.7283370494842529),
 ('흥미', 0.7058372497558594),
 ('묘미', 0.6446568965911865),
 ('유쾌', 0.6441777348518372),
 ('재미있는', 0.6389391422271729),
 ('즐거운', 0.6228681206703186),
 ('톡톡', 0.6186307668685913),
 ('찾아보는', 0.6171889305114746),
 ('엉뚱함', 0.6071854829788208),
 ('유희', 0.6063868999481201)]

In [63]:
art2vec.wv.most_similar(['최순민','재미'],negative='김섭')

[('흥미', 0.4938831031322479),
 ('즐거움', 0.4859802722930908),
 ('막대사탕', 0.4640687108039856),
 ('디저트', 0.457018107175827),
 ('재밌는', 0.45514976978302),
 ('재미있는', 0.45226866006851196),
 ('힌트', 0.44326353073120117),
 ('아기자기', 0.43784695863723755),
 ('재미있고', 0.4362080693244934),
 ('예쁜', 0.4261799454689026)]

In [60]:
art2vec.wv.most_similar('김혜연')

[('임만혁', 0.7648429870605469),
 ('박형진', 0.7506508827209473),
 ('김덕기', 0.7272018194198608),
 ('이상선', 0.725709080696106),
 ('김태우', 0.7161332964897156),
 ('서은', 0.707787036895752),
 ('박지혜', 0.7039687633514404),
 ('최순민', 0.7037951946258545),
 ('손민광', 0.7032536864280701),
 ('김은기', 0.7019984722137451)]

In [65]:
art2vec.wv.most_similar('임만혁','김혜연')

[('정작', 0.39441823959350586),
 ('어슷비슷', 0.3715914785861969),
 ('만이', 0.36983853578567505),
 ('명백', 0.35792189836502075),
 ('창조할', 0.3490333557128906),
 ('무기', 0.34765177965164185),
 ('생산하지', 0.3454466462135315),
 ('무너짐', 0.34017521142959595),
 ('음에도', 0.33978211879730225),
 ('으론', 0.3387604355812073)]

artistsDoc to Vec

In [2]:
with open('artists.lst','rb') as file:
    artists_lst = pickle.load(file)

In [3]:
documents = [TaggedDocument(artist[1], artist[0]) for artist in artists_lst]

In [33]:
max_epochs = 10
vec_size = 100
alpha = 0.025

In [31]:
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

In [32]:
model.build_vocab(documents)

In [34]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_artist.model")
print("Model Saved")

iteration 0


C:\Users\Chankoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Model Saved


In [26]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [10]:
tw = Twitter()

In [11]:
query_tokens = tw.morphs(query)

In [35]:
new_vector = model.infer_vector(query_tokens,steps=100) #??
sims = model.docvecs.most_similar([new_vector])
sims

[('준', 0.5818822383880615),
 ('오', 0.5476326942443848),
 ('림', 0.5334258079528809),
 ('남', 0.5329720377922058),
 ('환', 0.5254910588264465),
 ('일', 0.524368405342102),
 ('용', 0.5242956876754761),
 ('관', 0.5224420428276062),
 ('수', 0.5215528607368469),
 ('상', 0.5158921480178833)]